In [ ]:
!pip install transformers datasets

#Download and explore data

In [ ]:
from datasets import load_dataset
#Tải dữ liệu squad cho QA từ datasets
raw_ds = load_dataset('squad')
raw_ds

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
#Mẫu đàu tiên trong tập train
raw_ds['train'][1]

{'id': '5733be284776f4190066117f',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is in front of the Notre Dame Main Building?',
 'answers': {'text': ['a copper statue of Christ'], 'answer_start': [188]}}

In [ ]:
raw_ds['train'][-2]

{'id': '5735d259012e2f140011a0a0',
 'title': 'Kathmandu',
 'context': "Kathmandu Metropolitan City (KMC), in order to promote international relations has established an International Relations Secretariat (IRC). KMC's first international relationship was established in 1975 with the city of Eugene, Oregon, United States. This activity has been further enhanced by establishing formal relationships with 8 other cities: Motsumoto City of Japan, Rochester of the USA, Yangon (formerly Rangoon) of Myanmar, Xi'an of the People's Republic of China, Minsk of Belarus, and Pyongyang of the Democratic Republic of Korea. KMC's constant endeavor is to enhance its interaction with SAARC countries, other International agencies and many other major cities of the world to achieve better urban management and developmental programs for Kathmandu.",
 'question': 'In what year did Kathmandu create its initial international relationship?',
 'answers': {'text': ['1975'], 'answer_start': [199]}}

In [ ]:
#Kiểm tra danh sách answers trong tập train có list nào > 1 không
raw_ds['train'].filter(lambda x: len(x['answers']['text']) != 1)

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

In [ ]:
#Mẫu trong tập validation
raw_ds['validation'][1]

{'id': '56be4db0acb8001400a502ed',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the NFC at Super Bowl 50?',
 'answers': {'text': ['Carolina Panthers',
   'Carolina Panthers',
   'Caroli

# Using tokenizer

In [ ]:
from transformers import AutoTokenizer
#Tạo tokenizer từ AutoTokenizer của pretrained model
model_checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
#Test tokenizer
contexts = raw_ds['train'][1]['context']
questions = raw_ds['train'][1]['question']

inputs = tokenizer(
    questions, 
    contexts,
    max_length=100, #Chỉ định số lượng mã thông báo đầu vào nên dài 100 token
    truncation='only_second', #Chỉ cắt bớt ngữ cảnh
    stride=50, #Bước nhảy để chia các windows
    return_overflowing_tokens=True, #Trả về các tokens trong các phần chồng chéo của các context windows 
    return_offsets_mapping=True, #
)

inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [ ]:
#Chuyển input_ids thành string
for ids in inputs['input_ids']:
  print(tokenizer.decode(ids))

[CLS] What is in front of the Notre Dame Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the G [SEP]
[CLS] What is in front of the Notre Dame Main Building? [SEP] facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernade [SEP]
[CLS] What is in front of the Notre Dame Main Building? [SEP] of the Sacred Heart. Immediately behind the basilica is the Grotto, 

In [ ]:
#Chỉ ra các đầu ra đến từ đâu. VD 4 id đầu tiên đến từ mẫu ban đầu có index là 0 trong tập dữ liệu đầu vào
inputs['overflow_to_sample_mapping']

[0, 0, 0, 0]

In [ ]:
# offset_mapping cho biết vị trí của mỗi token trong toàn bộ chuỗi đầu vào nối giữa question và context
# Lưu ý:
# token đặt biệt(CLS, SEP) sẽ chiếm 0 vị trí nên nó được đăt là (0,0)
# Phần câu hỏi là giống nhau cho mỗi mẫu 
# Điểm băt đầu của phần bối cảnh tăng lên trong mỗi mẫu 
inputs['offset_mapping']

[[(0, 0),
  (0, 4),
  (5, 7),
  (8, 10),
  (11, 16),
  (17, 19),
  (20, 23),
  (24, 29),
  (30, 34),
  (35, 39),
  (40, 48),
  (48, 49),
  (0, 0),
  (0, 13),
  (13, 15),
  (15, 16),
  (17, 20),
  (21, 27),
  (28, 31),
  (32, 33),
  (34, 42),
  (43, 52),
  (52, 53),
  (54, 56),
  (56, 58),
  (59, 62),
  (63, 67),
  (68, 76),
  (76, 77),
  (77, 78),
  (79, 83),
  (84, 88),
  (89, 91),
  (92, 93),
  (94, 100),
  (101, 107),
  (108, 110),
  (111, 114),
  (115, 121),
  (122, 126),
  (126, 127),
  (128, 139),
  (140, 142),
  (143, 148),
  (149, 151),
  (152, 155),
  (156, 160),
  (161, 169),
  (170, 173),
  (174, 180),
  (181, 183),
  (183, 184),
  (185, 187),
  (188, 189),
  (190, 196),
  (197, 203),
  (204, 206),
  (207, 213),
  (214, 218),
  (219, 223),
  (224, 226),
  (226, 229),
  (229, 232),
  (233, 237),
  (238, 241),
  (242, 248),
  (249, 250),
  (250, 251),
  (251, 254),
  (254, 256),
  (257, 259),
  (260, 262),
  (263, 264),
  (264, 265),
  (265, 268),
  (268, 269),
  (269, 270),
 

In [ ]:
#None biểu thị token đặt biệt
#0 biểu thị token thuộc về question
#1 biểu thị token thuộc về context
print(inputs.sequence_ids(1))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [ ]:
#Tìm kiếm vị trí bắt đầu và kết thúc cho context của mẫu ví dụ đầu tiên
sequence_ids = inputs.sequence_ids(1)

ctx_start = sequence_ids.index(1) 
ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

ctx_start, ctx_end

(13, 98)

In [ ]:
#Tính toán lại vị trí phù hợp của answer trong chuỗi token được chia, từ vị trí của answer_start trong contex đầy đủ

#Xem xét cho mẫu ví dụ 1
answer = raw_ds['train'][1]['answers']

ans_start_char = answer['answer_start'][0]
ans_end_char = ans_start_char + len(answer['text'][0])

offset = inputs['offset_mapping'][0]

start_idx = 0
end_idx = 0

#Kiểm tra câu trả lời có vược quá context window hay không
if offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char:
  print('target is (0,0)')
else:
  #Tìm kiếm vị trí bắt đầu và kết thúc của token
  i = ctx_start
  for start_end_char in offset[ctx_start:]:
    start, end = start_end_char
    if start == ans_start_char:
      start_idx = i
    if end == ans_end_char:
      end_idx = i
      break
    i += 1 

start_idx, end_idx

(53, 57)

In [ ]:
input_ids = inputs['input_ids'][0]
tokenizer.decode(input_ids[start_idx: end_idx+1])

'a copper statue of Christ'

In [ ]:
def find_answer_token_idx(
    ctx_start,
    ctx_end,
    ans_start_char,
    ans_end_char,
    offset):
  
  start_idx = 0
  end_idx = 0

  if offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char:
    pass
  else:
    i = ctx_start
    for start_end_char in offset[ctx_start:]:
      start, end = start_end_char
      if start == ans_start_char:
        start_idx = i      
      if end == ans_end_char:
        end_idx = i
        break
      i += 1
  return start_idx, end_idx

In [ ]:
#Thử với tất cả context windows

start_idxs = []
end_idxs = []

for i, offset in enumerate(inputs['offset_mapping']):
  sequence_ids = inputs.sequence_ids(i)

  #Tìm kiếm ctx_start và ctx_end
  ctx_start = sequence_ids.index(1)
  ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

  start_idx, end_idx = find_answer_token_idx(
      ctx_start,
      ctx_end,
      ans_start_char,
      ans_end_char,
      offset
  )

  start_idxs.append(start_idx)
  end_idxs.append(end_idx)

start_idxs, end_idxs

([53, 17, 0, 0], [57, 21, 0, 0])

In [ ]:
max_length = 384
stride = 128

def tokenize_fn_train(batch):
  # Xóa các khoảng trắng ở hai đầu trong câu hỏi
  questions = [q.strip() for q in batch["question"]]

  # Thực hiện tokenize trên batch
  inputs = tokenizer(
    questions,
    batch["context"],
    max_length=max_length,
    truncation="only_second",
    stride=stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
  )

  # we don't need these later so remove them
  offset_mapping = inputs.pop("offset_mapping")
  orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
  answers = batch['answers']
  start_idxs, end_idxs = [], []


  for i, offset in enumerate(offset_mapping):
    sample_idx = orig_sample_idxs[i]
    answer = answers[sample_idx]

    ans_start_char = answer['answer_start'][0]
    ans_end_char = ans_start_char + len(answer['text'][0])

    sequence_ids = inputs.sequence_ids(i)

    ctx_start = sequence_ids.index(1)
    ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

    start_idx, end_idx = find_answer_token_idx(
      ctx_start,
      ctx_end,
      ans_start_char,
      ans_end_char,
      offset)

    start_idxs.append(start_idx)
    end_idxs.append(end_idx)
  
  inputs["start_positions"] = start_idxs
  inputs["end_positions"] = end_idxs
  return inputs

In [ ]:
train_ds = raw_ds['train'].map(
    tokenize_fn_train,
    batched=True,
    remove_columns=raw_ds['train'].column_names
)

len(train_ds)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

88729

In [ ]:
len(raw_ds['train'])

87599

In [ ]:
raw_ds['validation'][0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


In [ ]:
def tokenize_fn_validation(batch):
  # some questions have leading and/or trailing whitespace
  questions = [q.strip() for q in batch["question"]]

  # tokenize the data (with padding this time)
  # since most contexts are long, we won't bother to pad per-minibatch
  inputs = tokenizer(
    questions,
    batch["context"],
    max_length=max_length,
    truncation="only_second",
    stride=stride,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    padding="max_length",
  )

  # we don't need these later so remove them
  orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
  sample_ids = []

  # rewrite offset mapping by replacing question tuples with None
  # this will be helpful later on when we compute metrics
  for i in range(len(inputs["input_ids"])):
    sample_idx = orig_sample_idxs[i]
    sample_ids.append(batch['id'][sample_idx])

    sequence_ids = inputs.sequence_ids(i)
    offset = inputs["offset_mapping"][i]
    inputs["offset_mapping"][i] = [
      x if sequence_ids[j] == 1 else None for j, x in enumerate(offset)]
    
  inputs['sample_id'] = sample_ids
  return inputs

In [ ]:
validation_ds = raw_ds['validation'].map(
    tokenize_fn_validation,
    batched=True,
    remove_columns=raw_ds['validation'].column_names
)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
len(raw_ds['validation']), len(validation_ds)

(10570, 10822)

#Metric

In [ ]:
from datasets import load_metric

metric = load_metric('squad')

<ipython-input-26-201cb45a6bb4>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('squad')


In [ ]:
predicted_answers = [
    {'id': '1', 'prediction_text': 'Albert Einstein'},
    {'id': '2', 'prediction_text': 'physicist'},
    {'id': '3', 'prediction_text': 'general relativity'}
]

true_answes = [
    {'id': '1', 'answers': {'text': ['Albert Einstein'], 'answer_start': [100]}},
    {'id': '2', 'answers': {'text': ['physicist'], 'answer_start': [100]}},
    {'id': '3', 'answers': {'text': ['special relativity'], 'answer_start': [100]}}
]

metric.compute(predictions=predicted_answers, references=true_answes)

{'exact_match': 66.66666666666667, 'f1': 83.33333333333333}

In [ ]:
from tqdm.autonotebook import tqdm

n_largest = 20
max_answer_length = 30
predicted_answers = []

def compute_metrics(start_logits, end_logits, processed_dataset, orig_dataset):
  # map sample_id ('56be4db0acb8001400a502ec') to row indices of processed data
  sample_id2idxs = {}
  for i, id_ in enumerate(processed_dataset['sample_id']):
    if id_ not in sample_id2idxs:
      sample_id2idxs[id_] = [i]
    else:
      sample_id2idxs[id_].append(i)

  predicted_answers = []
  for sample in tqdm(orig_dataset):

    sample_id = sample['id']
    context = sample['context']

    # update these as we loop through candidate answers
    best_score = float('-inf')
    best_answer = None

    # now loop through the *expanded* input samples (fixed size context windows)
    # from here we will pick the highest probability start/end combination
    for idx in sample_id2idxs[sample_id]:
      start_logit = start_logits[idx] # (T,) vector
      end_logit = end_logits[idx] # (T,) vector

      # note: do NOT do the reverse: ['offset_mapping'][idx]
      offsets = processed_dataset[idx]['offset_mapping']

      start_indices = (-start_logit).argsort()
      end_indices = (-end_logit).argsort()

      for start_idx in start_indices[:n_largest]:
        for end_idx in end_indices[:n_largest]:

          # skip answers not contained in context window
          # recall: we set entries not pertaining to context to None earlier
          if offsets[start_idx] is None or offsets[end_idx] is None:
            continue
          
          # skip answers where end < start
          if end_idx < start_idx:
            continue
          
          # skip answers that are too long
          if end_idx - start_idx + 1 > max_answer_length:
            continue
          
          # see theory lecture for score calculation
          score = start_logit[start_idx] + end_logit[end_idx]
          if score > best_score:
            best_score = score

            # find positions of start and end characters
            # recall: offsets contains tuples for each token:
            # (start_char, end_char)
            first_ch = offsets[start_idx][0]
            last_ch = offsets[end_idx][1]

            best_answer = context[first_ch:last_ch]

    # save best answer
    predicted_answers.append({'id': sample_id, 'prediction_text': best_answer})
  
  # compute the metrics
  true_answers = [
    {'id': x['id'], 'answers': x['answers']} for x in orig_dataset
  ]
  return metric.compute(predictions=predicted_answers, references=true_answers)

#Train and Evaluate

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    'finetuned-squad',
    evaluation_strategy='no',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True
)

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=validation_ds,
    tokenizer=tokenizer
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,3.272000
1000,2.249000
1500,2.005700
2000,1.838000
2500,1.654000
3000,1.551700
3500,1.545700
4000,1.495500
4500,1.478400
5000,1.357200


TrainOutput(global_step=33276, training_loss=1.050382594647266, metrics={'train_runtime': 4198.0581, 'train_samples_per_second': 63.407, 'train_steps_per_second': 7.927, 'total_flos': 2.608361755366349e+16, 'train_loss': 1.050382594647266, 'epoch': 3.0})

In [ ]:
trainer_output = trainer.predict(validation_ds)

In [ ]:
trainer_output

PredictionOutput(predictions=(array([[ -6.484, -10.29 , -10.92 , ..., -11.695, -11.67 , -11.75 ],
       [ -5.86 , -10.266, -10.98 , ..., -11.695, -11.68 , -11.76 ],
       [ -7.684,  -9.805, -10.78 , ..., -11.62 , -11.63 , -11.69 ],
       ...,
       [ -5.66 ,  -9.92 , -10.82 , ..., -11.72 , -11.7  , -11.73 ],
       [ -5.74 , -10.56 , -11.125, ..., -11.71 , -11.695, -11.62 ],
       [ -5.035, -10.52 , -10.91 , ..., -11.766, -11.75 , -11.766]],
      dtype=float16), array([[ -5.547, -10.77 , -10.22 , ..., -11.41 , -11.445, -11.38 ],
       [ -5.008, -10.695, -10.14 , ..., -11.39 , -11.43 , -11.37 ],
       [ -6.605, -10.98 , -11.164, ..., -11.54 , -11.54 , -11.49 ],
       ...,
       [ -4.824, -11.19 , -11.16 , ..., -11.56 , -11.59 , -11.555],
       [ -4.734, -11.31 , -11.016, ..., -11.53 , -11.57 , -11.64 ],
       [ -4.11 , -11.38 , -11.47 , ..., -11.54 , -11.56 , -11.54 ]],
      dtype=float16)), label_ids=None, metrics={'test_runtime': 45.0624, 'test_samples_per_second': 240.15

In [ ]:
predictions, _, _ = trainer_output
predictions

(array([[ -6.484, -10.29 , -10.92 , ..., -11.695, -11.67 , -11.75 ],
        [ -5.86 , -10.266, -10.98 , ..., -11.695, -11.68 , -11.76 ],
        [ -7.684,  -9.805, -10.78 , ..., -11.62 , -11.63 , -11.69 ],
        ...,
        [ -5.66 ,  -9.92 , -10.82 , ..., -11.72 , -11.7  , -11.73 ],
        [ -5.74 , -10.56 , -11.125, ..., -11.71 , -11.695, -11.62 ],
        [ -5.035, -10.52 , -10.91 , ..., -11.766, -11.75 , -11.766]],
       dtype=float16),
 array([[ -5.547, -10.77 , -10.22 , ..., -11.41 , -11.445, -11.38 ],
        [ -5.008, -10.695, -10.14 , ..., -11.39 , -11.43 , -11.37 ],
        [ -6.605, -10.98 , -11.164, ..., -11.54 , -11.54 , -11.49 ],
        ...,
        [ -4.824, -11.19 , -11.16 , ..., -11.56 , -11.59 , -11.555],
        [ -4.734, -11.31 , -11.016, ..., -11.53 , -11.57 , -11.64 ],
        [ -4.11 , -11.38 , -11.47 , ..., -11.54 , -11.56 , -11.54 ]],
       dtype=float16))

In [ ]:
start_logits, end_logits = predictions

In [ ]:
len(predictions[0])

10822

In [ ]:
raw_ds['validation']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 10570
})

In [ ]:
compute_metrics(
    start_logits,
    end_logits,
    validation_ds,
    raw_ds['validation']
)

  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 77.24692526017029, 'f1': 85.56248742090618}

#Save and using model

In [ ]:
trainer.save_model('my_saved_model')

In [ ]:
from transformers import pipeline

qa = pipeline(
    'question-answering',
    model='my_saved_model',
)

In [ ]:
context = 'Today I went to the store to purchase a carton of milk.'
question = 'What did I buy?'

qa(context=context, question=question)

{'score': 0.6552767157554626,
 'start': 38,
 'end': 54,
 'answer': 'a carton of milk'}

In [ ]:
context = 'I will write a reply to the email interview today'
question = 'what will i do today?'
qa(context=context, question=question)

{'score': 0.7979496717453003,
 'start': 7,
 'end': 43,
 'answer': 'write a reply to the email interview'}

In [ ]:
qa(context='I want to eat fried rice', question='what will i eat?')

{'score': 0.9601355791091919, 'start': 14, 'end': 24, 'answer': 'fried rice'}